In [77]:
# Imported Python files do not automatically reload after you modify them
# M1: Automatically Reload
%load_ext autoreload
%autoreload 2
# M2: Manual Reload
# import importlib
# import ksfdtw
# importlib.reload(ksfdtw)

# https://stackoverflow.com/questions/20309456/how-do-i-call-a-function-from-another-py-file
from ksfdtw import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tslearn.metrics import dtw, lb_keogh
from sklearn.metrics import accuracy_score
from tslearn.neighbors import KNeighborsTimeSeriesClassifier
from tslearn.metrics import dtw as tsln_dtw
from tqdm import tqdm

/Users/cyuab/miniconda3/envs/ksfdtw/lib/python3.12/site-packages/tslearn/bases/bases.py:15: UserWarning: h5py not installed, hdf5 features will not be supported.
Install h5py to use hdf5 features: http://docs.h5py.org/
  warn(h5py_msg)


In [ ]:
# # A neat way to load the dataset
# data = np.load("../data/gunpoint_preprocessed.npz")
# data_dict = {key: data[key] for key in data.files}


In [3]:
# A old way to load the dataset
data = np.load("../data/gunpoint_preprocessed.npz", allow_pickle=True)
X_train_scaled = data["X_train_scaled"]
X_train_proc = data["X_train_proc"]
X_train_proc_noisy = data["X_train_proc_noisy"]
y_train = data["y_train"]
X_test_scaled = data["X_test_scaled"]
X_test_proc = data["X_test_proc"]
X_test_proc_noisy = data["X_test_proc_noisy"]
y_test = data["y_test"]
train_cutting_orig = data["train_cutting_orig"].tolist()
train_cutting_final = data["train_cutting_final"].tolist()
test_cutting_orig = data["test_cutting_orig"].tolist()
test_cutting_final = data["test_cutting_final"].tolist()


In [4]:
# Compute Euclidean distances to all training samples
def euclidean_distance(a, b):
    return np.sqrt(np.sum((a - b) ** 2))

In [ ]:
m = len(X_train_proc[0])
n = len(X_train_scaled[0])
l = 2
L = int(np.floor(min(np.ceil(l*m),n)/3))

In [8]:
# ps
results = []
for i in range(0,50): # X_train_proc.shape[0]
    results.append([ps_distance_p(X_train_proc[i], x, 2, 0.1, 10, 3, distance_method='dtw', lower_bound_method=lb_kim_fl) for x in X_train_scaled[:50]])
np.savez("../results/results.npz", results=np.array(results, dtype=object))


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


In [84]:
len(results_noisy)

50

In [ ]:
# ps + noise
results_noisy = []
for i in range(0,50): # X_train_proc.shape[0]
    results_noisy.append([ps_distance_p(X_train_proc_noisy[i], x, 2, 0.1, 10, 3, distance_method='dtw', lower_bound_method=lb_kim_fl) for x in X_train_scaled])
np.savez("../results/results_noisy.npz", results=np.array(results_noisy, dtype=object))

In [48]:
# ps
# Check the pruning power
results_lb_shen = []

for i in range(0,10): # X_train_proc.shape[0]
    results_lb_shen.append([ps_distance_p(X_train_proc[i], x, 2, 0.1, 10, 3, distance_method='dtw', lower_bound_method=lb_shen) for x in X_train_scaled[:10]])
    print(i)
np.savez("../results/results_lb_shen.npz", results=np.array(results_lb_shen, dtype=object))


0
1
2
3
4
5
6
7
8
9


In [71]:
# ps + noise
# Check the pruning power
results_lb_shen2 = []

for i in range(0,10): # X_train_proc.shape[0]
    results_lb_shen2.append([ps_distance_p(X_train_proc_noisy[i], x, 2, 0.1, 10, 3, distance_method='dtw', lower_bound_method=lb_shen) for x in X_train_scaled[:10]])
    print(i)
np.savez("../results/results_lb_shen.npz", results=np.array(results_lb_shen2, dtype=object))


0
1
2
3
4
5
6
7
8
9


In [60]:
# ps
results_no_prune = []
for i in range(0,10): # X_train_proc.shape[0]
    results_no_prune.append([ps_distance_p_without_prune(X_train_proc[i], x, 2, 0.1, 10, 3, distance_method='dtw', lower_bound_method=lb_dummy) for x in X_train_scaled[:10]])
    print(i)
np.savez("../results/results_no_prune.npz", results=np.array(results_no_prune, dtype=object))

0
1
2
3
4
5
6
7
8
9


In [70]:
# ps + noise
results_no_prune2 = []
for i in range(0,10): # X_train_proc.shape[0]
    results_no_prune2.append([ps_distance_p_without_prune(X_train_proc_noisy[i], x, 2, 0.1, 10, 3, distance_method='dtw', lower_bound_method=lb_dummy) for x in X_train_scaled[:10]])
    print(i)
np.savez("../results/results_no_prune2.npz", results=np.array(results_no_prune2, dtype=object))

0
1
2
3
4
5
6
7
8
9


In [67]:
# ps
# No prune
total_no_iterations =0
for i in range(0,10):
     # cuts = [r[1] for r in results[i]]
     iterations = np.array([r[2] for r in results_no_prune[i]])
     total_no_iterations += sum(iterations)  
     # best_idx = np.argmin(distances)
     # if i != best_idx:
     #      print(i, "cannot retrieve the original time series")
print(total_no_iterations)
total_no_iterations_no_prune = total_no_iterations
        

3985237


In [72]:
# ps + noise
# No prune
total_no_iterations =0
for i in range(0,10):
     # cuts = [r[1] for r in results[i]]
     iterations = np.array([r[2] for r in results_no_prune2[i]])
     total_no_iterations += sum(iterations)  
     # best_idx = np.argmin(distances)
     # if i != best_idx:
     #      print(i, "cannot retrieve the original time series")
print(total_no_iterations)
total_no_iterations_no_prune = total_no_iterations
        

4023811


In [66]:
# ps
# Using LB_Shen
total_no_iterations =0
for i in range(0,10):
     # cuts = [r[1] for r in results[i]]
     iterations = np.array([r[2] for r in results_lb_shen[i]])
     total_no_iterations += sum(iterations)  
     # best_idx = np.argmin(distances)
     # if i != best_idx:
     #      print(i, "cannot retrieve the original time series")
print(total_no_iterations)
total_no_iterations_lb_shen = total_no_iterations

1404294


In [73]:
# ps + noise
# Using LB_Shen
total_no_iterations =0
for i in range(0,10):
     # cuts = [r[1] for r in results[i]]
     iterations = np.array([r[2] for r in results_lb_shen2[i]])
     total_no_iterations += sum(iterations)  
     # best_idx = np.argmin(distances)
     # if i != best_idx:
     #      print(i, "cannot retrieve the original time series")
print(total_no_iterations)
total_no_iterations_lb_shen = total_no_iterations

1483516


In [76]:
(total_no_iterations_no_prune - total_no_iterations_lb_shen)/total_no_iterations_no_prune

np.float64(0.6313156855528255)

# Precision

In [21]:
def precision_at_k(distances, true_index, k):
    # Get the indices of the top-k smallest distances
    top_k_indices = sorted(range(len(distances)), key=lambda x: distances[x])[:k]
    
    # Check if the true match is among them
    return 1 if true_index in top_k_indices else 0


In [32]:
# PSDTW
precision_at_1, precision_at_3, precision_at_5, precision_at_7 = 0, 0, 0, 0
for i in range(0,50):
     distances = np.array([r[0] for r in results[i]])
     precision_at_1 += precision_at_k(distances, i, 1)
     precision_at_3 += precision_at_k(distances, i, 3)
     precision_at_5 += precision_at_k(distances, i, 5)
     precision_at_7 += precision_at_k(distances, i, 7)
     # cuts = [r[1] for r in results[i]]
     # iterations = np.array([r[2] for r in results[i]])  
     # best_idx = np.argmin(distances)
     # if i != best_idx:
     #      print(i, "cannot retrieve the original time series")
print(precision_at_1/50, precision_at_3/50, precision_at_5/50, precision_at_7/50) 
        

0.8 0.96 1.0 1.0


In [33]:
# PSDTW, noisy
precision_at_1, precision_at_3, precision_at_5, precision_at_7 = 0, 0, 0, 0
for i in range(0,50):
     distances = np.array([r[0] for r in results_noisy[i]])
     precision_at_1 += precision_at_k(distances, i, 1)
     precision_at_3 += precision_at_k(distances, i, 3)
     precision_at_5 += precision_at_k(distances, i, 5)
     precision_at_7 += precision_at_k(distances, i, 7)
     # cuts = [r[1] for r in results[i]]
     # iterations = np.array([r[2] for r in results[i]])  
     # best_idx = np.argmin(distances)
     # if i != best_idx:
     #      print(i, "cannot retrieve the original time series")
print(precision_at_1/50, precision_at_3/50, precision_at_5/50, precision_at_7/50) 

0.48 0.82 0.96 1.0


In [34]:
# DTW
precision_at_1, precision_at_3, precision_at_5, precision_at_7 = 0, 0, 0, 0
for i in range(0,50):
    distances = np.array([tsln_dtw(X_train_proc[i], x) for x in X_train_scaled])
    precision_at_1 += precision_at_k(distances, i, 1)
    precision_at_3 += precision_at_k(distances, i, 3)
    precision_at_5 += precision_at_k(distances, i, 5)
    precision_at_7 += precision_at_k(distances, i, 7)
print(precision_at_1, precision_at_3, precision_at_5, precision_at_7)
print(precision_at_1/50, precision_at_3/50, precision_at_5/50, precision_at_7/50)  

24 37 43 45
0.48 0.74 0.86 0.9


In [35]:
# DTW Noisy
precision_at_1, precision_at_3, precision_at_5, precision_at_7 = 0, 0, 0, 0
for i in range(0,50):
    distances = np.array([tsln_dtw(X_train_proc_noisy[i], x) for x in X_train_scaled])
    precision_at_1 += precision_at_k(distances, i, 1)
    precision_at_3 += precision_at_k(distances, i, 3)
    precision_at_5 += precision_at_k(distances, i, 5)
    precision_at_7 += precision_at_k(distances, i, 7)
print(precision_at_1, precision_at_3, precision_at_5, precision_at_7)
print(precision_at_1/50, precision_at_3/50, precision_at_5/50, precision_at_7/50)  

22 32 39 43
0.44 0.64 0.78 0.86


In [36]:
# ED
precision_at_1, precision_at_3, precision_at_5, precision_at_7 = 0, 0, 0, 0
for i in range(0,50):
    distances = np.array([euclidean_distance(X_train_proc[i], x) for x in X_train_scaled])
    precision_at_1 += precision_at_k(distances, i, 1)
    precision_at_3 += precision_at_k(distances, i, 3)
    precision_at_5 += precision_at_k(distances, i, 5)
    precision_at_7 += precision_at_k(distances, i, 7)
print(precision_at_1, precision_at_3, precision_at_5, precision_at_7)
print(precision_at_1/50, precision_at_3/50, precision_at_5/50, precision_at_7/50)  

1 3 5 9
0.02 0.06 0.1 0.18


In [41]:
# ED
precision_at_1, precision_at_3, precision_at_5, precision_at_7 = 0, 0, 0, 0
for i in range(0,50):
    distances = np.array([euclidean_distance(X_train_proc_noisy[i], x) for x in X_train_scaled])
    precision_at_1 += precision_at_k(distances, i, 1)
    precision_at_3 += precision_at_k(distances, i, 3)
    precision_at_5 += precision_at_k(distances, i, 5)
    precision_at_7 += precision_at_k(distances, i, 7)
print(precision_at_1, precision_at_3, precision_at_5, precision_at_7)
print(precision_at_1/50, precision_at_3/50, precision_at_5/50, precision_at_7/50)  

1 3 4 9
0.02 0.06 0.08 0.18


In [ ]:
# def test_querying_ed_dtw(query_ts_idx, dist_method):
#     query_ts = X_train_proc_noisy[query_ts_idx]
#     query_ts_original = X_train[query_ts_idx]
#     if dist_method == "dtw":
#         distances = np.array([tsln_dtw(query_ts, x) for x in X_train])
#     elif dist_method == "psdtw":
#         results = [ps_distance_p(query_ts, x, 2, 0.1, L, 3) for x in X_train[:5]]
#         distances = np.array([r[0] for r in results])
#         cuts = [r[1] for r in results]
#         iterations = np.array([r[2] for r in results])  
#     else:
#         raise ValueError("Must be either dtw or psdtw")
#     # Find best match
#     best_idx = np.argmin(distances)
#     best_match = X_train[best_idx]

#     print(f"Best match index: {best_idx}")
#     print(f"Distance: {distances[best_idx]:.4f}")
#     plt.plot(query_ts, label="Query", linewidth=2, color= "red")
#     plt.plot(query_ts_original, label="Original version of Query", linestyle="--", color="green")
#     plt.plot(best_match, label="Best Match", linestyle="--", color="blue")

#     plt.legend()
#     plt.title("1-NN Query (Euclidean Distance)")
#     plt.show()

#     for i in range(0, 6):
#     test_querying_ed_dtw(i, "psdtw")



In [9]:
###

In [92]:
# cutting

In [93]:
data = np.load("../results/results.npz", allow_pickle=True)
results = data["results"]
data = np.load("../results/results_noisy.npz", allow_pickle=True)
results_noisy = data["results"]

In [96]:
len(results_noisy)

50

In [98]:
# PSDTW

for i in range(0,50):
     cuts = [r[1] for r in results[i]]
     print(cuts)



[[((0, 6), (0, 7)), ((6, 25), (7, 25)), ((25, 30), (25, 30))], [((0, 5), (0, 5)), ((5, 25), (5, 23)), ((25, 30), (23, 30))], [((0, 9), (0, 9)), ((9, 23), (9, 22)), ((23, 30), (22, 30))], [((0, 14), (0, 13)), ((14, 19), (13, 17)), ((19, 30), (17, 30))], [((0, 5), (0, 4)), ((5, 25), (4, 22)), ((25, 30), (22, 30))], [((0, 5), (0, 6)), ((5, 19), (6, 16)), ((19, 30), (16, 30))], [((0, 12), (0, 8)), ((12, 23), (8, 26)), ((23, 30), (26, 30))], [((0, 18), (0, 15)), ((18, 24), (15, 27)), ((24, 30), (27, 30))], [((0, 12), (0, 6)), ((12, 25), (6, 24)), ((25, 30), (24, 30))], [((0, 6), (0, 5)), ((6, 11), (5, 8)), ((11, 30), (8, 30))], [((0, 5), (0, 3)), ((5, 10), (3, 6)), ((10, 30), (6, 30))], [((0, 7), (0, 5)), ((7, 12), (5, 10)), ((12, 30), (10, 30))], [((0, 20), (0, 22)), ((20, 25), (22, 27)), ((25, 30), (27, 30))], [((0, 6), (0, 5)), ((6, 11), (5, 8)), ((11, 30), (8, 30))], [((0, 7), (0, 9)), ((7, 25), (9, 22)), ((25, 30), (22, 30))], [((0, 7), (0, 5)), ((7, 12), (5, 10)), ((12, 30), (10, 30))